# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [196]:
import pandas as pd
import numpy as np

## imports for Python, Pandas

In [197]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [198]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## JSON Mini Project Solutions

In [203]:
world_bank_projects_df = pd.read_json('data/world_bank_projects.json')

#### 1. Find the 10 countries with most projects

In [204]:
world_bank_projects_df.groupby('countryname').size().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64

#### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [205]:
#extract mjtheme_namecode column
project_themes_df = world_bank_projects_df[['mjtheme_namecode']]

#Convert dataframe to dict object to be used in json_normalize
project_themes_dict = project_themes_df.to_dict()

#Convert the dict of dicts to dataframe by using json_normalize
theme_names_df = pd.DataFrame()
for i in project_themes_dict['mjtheme_namecode']:
    theme_names_df = theme_names_df.append(json_normalize(project_themes_dict['mjtheme_namecode'][i]), ignore_index=True)

#Replace empty values under "name" column with NaN
theme_names_df = theme_names_df.replace(r'^\s*$', np.nan, regex=True)

#Groupby and get the top 10 major project themes, count() ignores NaN values
theme_names_df.groupby(['name']).name.count().sort_values(ascending=False).head(10)


name
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                           72 
Urban development                               47 
Economic management                             33 
Name: name, dtype: int64

#### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [206]:
#count the missing values in name column
print('Total number of missing names: %d' %theme_names_df['name'].isna().sum())

#sort and fill missing values with last valid "name" 
theme_names_df = theme_names_df.sort_values(['code', 'name']).ffill()
print('Total number of missing names after forward filling: %d' %theme_names_df['name'].isna().sum())

Total number of missing names: 122
Total number of missing names after forward filling: 0
